In [72]:
import pandas as pd
%pip install -Uq upgini catboost

In [113]:
df = pd.read_csv('train.csv')
df = df.sample(n=10_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(inplace=True,drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,9.0,20.0,28.0
1,2013-01-01,9.0,8.0,45.0
2,2013-01-01,2.0,4.0,21.0
3,2013-01-01,1.0,10.0,37.0
4,2013-01-01,4.0,2.0,30.0


In [74]:
from sklearn.model_selection import train_test_split

train = df[df['date'] < "2017-01-01"]
test = df[df['date'] >= '2017-01-10']

In [79]:
train_features = train.drop('sales', axis=1)
train_target = train['sales']

test_features = test.drop("sales", axis=1)
test_target = test['sales']

In [ ]:
%pip install upgini

# **Enrich the features**

In [112]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date" : SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,train_target, eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=79789725-a78b-4b04-aca1-3fecd6d15c55
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


48 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_48_b39cd0c4,0.0440,100.0000,"5.9023, 5.1738, 4.8806",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_31_fa6d9a99,0.0379,100.0000,"4.7167, 5.5458, 4.674",Upgini,Weather & climate normals data,Premium
f_autofe_div_b0093813,0.0221,100.0000,"-0.3893, -0.7264, 0.9401",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_mul_efd66bbe,0.0152,100.0000,"-0.2216, 1.3509, -0.781",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_0dcebab1,0.0104,100.0000,"1.0051, 1.0123, 1.0483",Upgini,AutoFE: features from Markets data,Free
f_autofe_mul_3f4a1860,0.0082,100.0000,"0.531, 0.2744, -0.6831",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_d6db5d7a,0.0079,100.0000,"0.827, 0.1153, 0.9933",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_mul_c256dd85,0.0051,100.0000,"0.4345, -0.7258, 0.4695",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_20a12630,0.0045,100.0000,"0.0072, 0.0301, -0.0086",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_mul_52933c24,0.0039,100.0000,"1.0306, 0.8921, 0.5986",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_3e22df83,0.0036,100.0000,"-0.0044, 0.0042, -0.0151",Upgini,"AutoFE: features from Calendar data,Markets data",Free


Upgini,"AutoFE: features from Calendar data,Markets data",0.0955,31
Upgini,Weather & climate normals data,0.0861,6
Upgini,AutoFE: features from Markets data,0.0195,7
Upgini,Calendar data,0.0007,1
Upgini,AutoFE: features from Calendar data,0.0007,2
Upgini,World economic indicators,0.0006,1


"Calendar data,Markets data",f_autofe_div_b0093813,f_events_date_week_sin1_847b5db1,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,/
"Calendar data,Markets data",f_autofe_mul_efd66bbe,f_events_date_year_cos1_9014a856,f_financial_date_vix_7d_to_1y_634c77eb,*
Markets data,f_autofe_div_0dcebab1,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_snp500_7d_to_1y_378b9f99,/
"Calendar data,Markets data",f_autofe_mul_3f4a1860,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_7d_to_7d_1y_shift_718136ef,*
"Calendar data,Markets data",f_autofe_div_d6db5d7a,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,/
"Calendar data,Markets data",f_autofe_mul_c256dd85,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,*
"Calendar data,Markets data",f_autofe_div_20a12630,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_mul_52933c24,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,*
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_mul_f5777239,f_events_date_week_sin1_847b5db1,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
Markets data,f_autofe_div_0b384b21,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,f_financial_date_silver_7d_to_1y_0ccfe462,/


We detected 73 outliers in your sample.
Examples of outliers with maximum value of target:
60    193.0
64    191.0
20    182.0
Name: target, dtype: float64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 18912
After dropping target outliers size: 18839
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15238,52.5340,289.2013,141.1643,148.0370
Eval 1,3674,61.6347,524.2339,389.2661,134.9679


In [104]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target, eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)


Preparing data...


[============================================================] 100% Finished


Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15238,52.8363,0.250353,0.155611,0.094743
1,Eval 1,3674,61.6347,0.261358,0.199518,0.061840


In [ ]:
enriched_train_features = enricher.transform(train_features,keep_input=True)
enriched_test_features = enricher.transform(test_features,keep_input=True)

[                                                            ] 0% Checking dataset...

You are trying to launch enrichment for 15238 rows, which will exceed the rest limit 10000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[=========================                                   ] 43% Enriching dataset...

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=a0af611b-bf80-4e2c-99d5-30e2ad71dc9c
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


In [102]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_preds,"SMAPE")

CatBoostError: ignored